In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *
%matplotlib inline

## Load data 

The columns represent the following:

buying - price of the car

maint - price of maintenance

doors - number of doors

persons - highest number of passengers that can be transported

lug_boot - size of luggage compartment/boot

safety - estimated safety of car

class - car acceptibility

In [4]:
data = 'data/car_evaluation.csv'
df = pd.read_csv(data, header=None)
# Rename the column
col_names = ['buying', 'meant', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df.columns = col_names

In [5]:
df.head()

,buying,meant,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   meant     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [10]:
for col in col_names:
    print(df[col].value_counts())

buying
vhigh    432
high     432
med      432
low      432
Name: count, dtype: int64
meant
vhigh    432
high     432
med      432
low      432
Name: count, dtype: int64
doors
2        432
3        432
4        432
5more    432
Name: count, dtype: int64
persons
2       576
4       576
more    576
Name: count, dtype: int64
lug_boot
small    576
med      576
big      576
Name: count, dtype: int64
safety
low     576
med     576
high    576
Name: count, dtype: int64
class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


In [ ]:
df.isnull().sum()

In [ ]:
df.head(5)

,buying,meant,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [14]:
for col in df.columns:
 print(col, " values ", df[col].unique())


buying  values  ['vhigh' 'high' 'med' 'low']
meant  values  ['vhigh' 'high' 'med' 'low']
doors  values  ['2' '3' '4' '5more']
persons  values  ['2' '4' 'more']
lug_boot  values  ['small' 'med' 'big']
safety  values  ['low' 'med' 'high']
class  values  ['unacc' 'acc' 'vgood' 'good']


In [15]:
X = df.drop(['class'], axis=1)
y = df['class']

In [ ]:
# Tabla de contingencia para la variable 'cost' y 'evaluation'
cost_vs_evaluation = pd.crosstab(df['buying'], df['class'])
print("\nDistribución de 'evaluation' según 'cost':")
print(cost_vs_evaluation)

# Repite esto para otras variables categóricas si es necesario

#### Data split

In [16]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33,
                                                    random_state=42)

X_train.shape, X_test.shape

((1157, 6), (571, 6))

In [26]:
# Encode Categorical
import category_encoders as ce
# encode variables with ordinal encoding
encoder = ce.OrdinalEncoder(cols=['buying', 'meant', 'doors', 'persons', 'lug_boot', 'safety'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

## Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=0)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [ ]:
from sklearn import tree
import graphviz 
dot_data = tree.export_graphviz(clf_gini, out_file=None, 
                              feature_names=X_train.columns,  
                              class_names=y_train,  
                              filled=True, rounded=True,  
                              special_characters=True)

graph = graphviz.Source(dot_data) 

graph 

## Eval the model

In [ ]:
# Check Accuracy score
from sklearn.metrics import accuracy_score

In [ ]:
y_pred_gini = clf_gini.predict(X_test)

In [ ]:
print("Model Accuracy score with criterion gini index {0:0.4f}"
      .format(accuracy_score(y_test, y_pred_gini)))

In [ ]:
y_pred_train_gini = clf_gini.predict(X_train)
y_pred_train_gini

In [ ]:
print("Model Accuracy score with criterion gini index {0:0.4f}"
      .format(accuracy_score(y_train, y_pred_train_gini)))

In [ ]:
print("Model Accuracy score with criterion gini index for test dataset {0:0.4f}"
      .format(accuracy_score(y_pred_gini, y_test)))
print("Model Accuracy score with criterion gini index for train dataset {0:0.4f}"
      .format(accuracy_score(y_pred_train_gini, y_train)))

Podemos observar que en nuestros datos de entrenamiento tenemos un alto porcentaje de *accuracy*.

Sin embargo, en los datos de prueba, el modelo no generaliza bien, ya que solo tenemos un menor porcentaje de *accuracy*. 

**Nuestro modelo está claramente sobreajustado.** 

Qué puede estar causando est esobreajuste? Cómo podríamos evitarlo?

#### Técnicas de poda: 

**Pre pruning**

La *poda previa* consiste en detener el crecimiento del árbol de decisión en una etapa temprana para evitar que se vuelva demasiado complejo y se ajuste en exceso a los datos de entrenamiento. 

Esto se logra estableciendo restricciones que limitan el crecimiento del árbol, como la profundidad máxima *(max_depth)*, el número mínimo de muestras para dividir un nodo *(min_samples_split)*, o el número mínimo de muestras en una hoja *(min_samples_leaf)*, entre otros.

Una estrategia eficaz para encontrar los mejores valores de estos parámetros es realizar una búsqueda en cuadrícula *(grid search)*. Este método prueba diferentes combinaciones de los parámetros y selecciona aquellos valores que logran el mejor rendimiento en los datos de prueba, asegurando un equilibrio entre precisión y generalización.

Controlaremos los siguientes hiperparámetros:

*max_depth:* Profundidad máxima del árbol de decisión.

*min_samples_split:* Número mínimo de muestras necesarias para dividir un nodo interno.

*min_samples_leaf:* Número mínimo de muestras necesarias para que un nodo sea considerado una hoja.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'max_depth': [2,4,6,8,10,12],
         'min_samples_split': [2,3,4],
         'min_samples_leaf': [1,2]}

clf = tree.DecisionTreeClassifier()
gcv = GridSearchCV(estimator=clf,param_grid=params)
gcv.fit(X_train,y_train)

In [ ]:
model = gcv.best_estimator_ # accuracy is the classifiers's default scoring method
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f'Train score {accuracy_score(y_train_pred,y_train)}')
print(f'Test score {accuracy_score(y_test_pred,y_test)}')

En este *notebook*, hemos utilizado *accuracy* como métrica de evaluación para medir el rendimiento del modelo. 
Sin embargo, como vimos durante el análisis exploratorio de datos, las clases en el conjunto de datos están desbalanceadas.

**Es *accuracy* una métrica adecuada en este caso?. Por qué?**

La precisión, aunque útil, no siempre es suficiente para entender el rendimiento del modelo. Esto es especialmente cierto en casos donde las clases están desbalanceadas (una clase tiene muchos más ejemplos que las otras). En estos casos, un modelo puede obtener una precisión alta simplemente prediciendo siempre la clase mayoritaria sin aprender a distinguir bien las clases minoritarias. 

*Por ejemplo*, en un problema de detección de fraudes donde el 99% de las transacciones son legítimas y solo el 1% son fraudulentas, un modelo que prediga que todas las transacciones son legítimas tendría una precisión del 99%, pero sería inútil para detectar fraudes.

**Qué métricas crees que sería apropiadas para el problema que estamos intentando resolver?**

En problemas de clasificación multiclase, existen diferentes formas de calcular métricas que reflejen el rendimiento general del modelo teniendo en cuenta el balance o importancia de cada clase. Estas son:

- *Macro*: Calcula la métrica de cada clase por separado y luego obtiene el promedio sin considerar la frecuencia de cada clase. Esto da igual peso a todas las clases, resaltando el rendimiento en clases minoritarias.

- *Weighted*: Promedia las métricas de cada clase ponderándolas según la frecuencia de la clase en el conjunto de datos. Es útil en problemas desbalanceados, ya que representa el rendimiento total del modelo de acuerdo con la distribución de las clases. Es una forma común de ajustar el impacto de las clases desbalanceadas, pero no es la única forma de ponderar en un modelo. Otras opcion puede ser una *ponderación personalizada*, donde se asignan pesos específicos a cada clase según su importancia relativa

- *Micro*: Considera cada predicción individual en lugar de calcular la métrica por clase. 

**Es suficiente utilizar métricas agregadas para evaluar y analizar el rendimiento de nuestro modelo?**

Para evaluar un modelo de clasificación en un problema multiclase es importante usar métricas que permitan analizar el desempeño por clase y no solo de forma agregada. Para nuestro modelo pueden haber clases más difíciles de clasificar que otras y ver las métricas de forma agregada no permite detectar dónde se están cometiendo los errores. 

La *matriz de confusión* es útil para observar dónde ocurren los errores específicos entre clases. 

Además, calcular *accuracy*, *recall* y *F1-Score* por clase ayuda a identificar en cuáles clases el modelo tiene un bajo rendimiento. 

Las *curvas ROC-AUC* por clase también pueden revelar la capacidad del modelo para distinguir entre clases en un esquema "one-vs-rest". 

Además, incluir el número de muestras por clase en los reportes es fundamental para contextualizar el rendimiento, ya que el desempeño suele variar en clases con menos ejemplos. 

Estos enfoques detallados ayudan a identificar sesgos hacia ciertas clases y a entender mejor el rendimiento del modelo.

In [ ]:
plot_confusionmatrix(y_pred_train_gini,y_train,dom='Train')
plot_confusionmatrix(y_pred_gini,y_test,dom='Test')

In [ ]:
from sklearn.metrics import classification_report

c_report = classification_report(y_test, y_pred_gini, target_names= X_test.columns.names)

c_report

#### Estudio individual

**Técnicas de poda posterior**

Existen varias técnicas de poda posterior, entre las cuales la poda por complejidad de costo es una de las más importantes.

*Poda por Complejidad de Costo*

Los árboles de decisión suelen sobreajustarse a los datos, es decir, capturan demasiado detalle en el conjunto de entrenamiento y luego tienen un bajo rendimiento en los datos de prueba. 
Una forma de evitarlo es limitar el crecimiento del árbol con restricciones (como max_depth y min_samples). 
Sin embargo, una técnica muy efectiva es la poda posterior a través de la *poda por complejidad de costo*, la cual mejora la precisión en los datos de prueba y genera un modelo más generalizable.

La poda por complejidad de costo se basa en encontrar el valor adecuado para el parámetro $\alpha$. 
Al probar diferentes valores de $\alpha$ en el árbol, podemos comparar la precisión de los árboles podados y seleccionar el valor que ofrezca el mejor equilibrio entre simplicidad y precisión en los datos de prueba.

Para profundizar más en la poda por complejidad de costo, puedes ver [este video de Josh Starmer.](https://www.youtube.com/watch?v=D0efHEJsfHo&list=PLblh5JKOoLUICTaGLRoHQDuF_7q2GfuJF&index=41)
